In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(0)
import datetime 
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import os
from catboost import CatBoostClassifier

import wget
from pathlib import Path
import shutil
import gzip

from matplotlib import pyplot as plt
%matplotlib inline

# Download ForestCoverType dataset

In [2]:
target = "Covertype"

bool_columns = [
    "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3",
    "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4",
    "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9",
    "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14",
    "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19",
    "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24",
    "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29",
    "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34",
    "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39",
    "Soil_Type40"
]

int_columns = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

feature_columns = (
    int_columns + bool_columns + [target])


# Simple preprocessing

Label encode categorical features and fill empty cells.

In [3]:
train = pd.read_csv("data/train_forest_cover.csv")
test = pd.read_csv("data/test_forest_cover.csv")

In [4]:
categorical_columns = []
categorical_dims =  {}
for col in train.columns[train.dtypes == object]:
    print(col, train[col].nunique())
    l_enc = LabelEncoder()
    train[col] = train[col].fillna("VV_likely")
    train[col] = l_enc.fit_transform(train[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

for col in train.columns[train.dtypes == 'float64']:
    train.fillna(train.mean(), inplace=True)
    
categorical_columns = []
categorical_dims =  {}
for col in test.columns[test.dtypes == object]:
    print(col, test[col].nunique())
    test[col] = test[col].fillna("VV_likely")
    test[col] = l_enc.transform(test[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

for col in test.columns[test.dtypes == 'float64']:
    test.fillna(test.mean(), inplace=True)

# Define categorical features for categorical embeddings

In [5]:
unused_feat = []

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


# Network parameters

# Training

# XGB

In [6]:
n_estimators = 5000 if not os.getenv("CI", False) else 20

In [7]:

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [8]:
X_train["Elevation"]

0         2631
1         3287
2         2130
3         3022
4         3190
          ... 
464804    2569
464805    2692
464806    3021
464807    3001
464808    2862
Name: Elevation, Length: 464809, dtype: int64

In [10]:
X_test

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2991,85,9,503,72,5124,233,225,124,5501,...,0,0,0,0,0,0,0,0,0,0
1,3360,185,13,566,102,4468,222,249,158,2561,...,0,0,1,0,0,0,0,0,0,0
2,3286,352,8,90,6,4097,206,226,158,1635,...,0,0,0,0,0,0,0,0,0,0
3,2861,103,13,607,29,450,242,222,108,1170,...,0,0,0,0,0,0,0,0,0,0
4,2834,97,9,108,13,1184,236,227,122,1832,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116198,2074,294,23,384,10,418,151,228,214,1025,...,0,0,0,0,0,0,0,0,0,0
116199,3120,281,11,182,0,1679,189,241,192,825,...,0,0,0,0,0,0,0,0,0,0
116200,3023,352,5,0,0,3129,211,231,158,361,...,0,0,0,0,0,0,0,0,0,0
116201,2858,248,5,30,0,3126,209,244,173,4052,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(max_depth=9,
    tree_method='gpu_hist',
    gpu_id=0,
    learning_rate=0.1,
    n_estimators=n_estimators,
    verbosity=0,
    silent=None,
    objective="multi:softmax",
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,)

clf_xgb.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=40,
            verbose=10)

/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:1.73470
[10]	validation_0-mlogloss:0.86831
[20]	validation_0-mlogloss:0.59127
[30]	validation_0-mlogloss:0.47581
[40]	validation_0-mlogloss:0.41868
[50]	validation_0-mlogloss:0.38148
[60]	validation_0-mlogloss:0.35854
[70]	validation_0-mlogloss:0.34067
[80]	validation_0-mlogloss:0.32827
[90]	validation_0-mlogloss:0.31882
[100]	validation_0-mlogloss:0.30877
[110]	validation_0-mlogloss:0.29856
[120]	validation_0-mlogloss:0.28761
[130]	validation_0-mlogloss:0.27696
[140]	validation_0-mlogloss:0.26820
[150]	validation_0-mlogloss:0.25694
[160]	validation_0-mlogloss:0.24879
[170]	validation_0-mlogloss:0.24067
[180]	validation_0-mlogloss:0.23127
[190]	validation_0-mlogloss:0.22359
[200]	validation_0-mlogloss:0.21629
[210]	validation_0-mlogloss:0.21020
[220]	validation_0-mlogloss:0.20483
[230]	validation_0-mlogloss:0.19865
[240]	validation_0-mlogloss:0.19332
[250]	validation_0-mlogloss:0.18866
[260]	validation_0-mlogloss:0.18434
[270]	validation_0-mlogloss:0.18049
[28

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=9, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=5000, n_jobs=-1,
              nthread=20, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=0, silent=None, subsample=0.7,
              tree_method='gpu_hist', ...)

In [12]:
preds_test = np.array(clf_xgb.predict_proba(X_test))
test_acc = accuracy_score(y_pred=np.argmax(preds_test, axis=1) + 1, y_true=y_test)
print(test_acc)

0.9714034921645741


In [13]:
pd.DataFrame(preds_test).to_csv('xgboost_preds.csv')


In [10]:
start_time = datetime.datetime.now()
X_train_scale = preprocessing.scale(X_train)
X_valid_scale = preprocessing.scale(X_test)
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train_scale, y_train)
print("SVC time execution(kernel='rbf' and C=1 ):",
     datetime.datetime.now() - start_time)


SVC time execution(kernel='rbf' and C=1 ): 0:16:38.391397


/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [31]:
p = np.array(clf.decision_function(X_valid_scale)) # decision is a voting function
prob = np.exp(p)/np.sum(np.exp(p),axis=1, keepdims=True) # softmax after the voting

In [32]:
pd.DataFrame(prob).to_csv('svm_preds_forest.csv')


In [16]:
from sklearn.metrics import log_loss
log_loss(y_test, prediction)

ValueError: y_true and y_pred contain different number of classes 7, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [1 2 3 4 5 6 7]

In [13]:
prediction = clf.predict(X_train_scale)
acc_score_train = accuracy_score(y_train, prediction)
prediction = clf.predict(X_valid_scale)
acc_score_valid = accuracy_score(y_test, prediction)
print("Train accuracy = {0}, Test accuracy = {1}".format(acc_score_train, acc_score_valid))

Train accuracy = 0.7121054024341181, Test accuracy = 0.7118490916757743


In [14]:
pd.DataFrame(prediction).to_csv('linear_svc.csv')

In [16]:
cat_params = {
    'iterations': 10000,
    'learning_rate': 0.03,
    'od_wait': 1000,
    'depth': 10,
    'task_type' : 'GPU',
    'l2_leaf_reg': 3,
    'eval_metric': 'MultiClass',
    'devices' : '0',
    'verbose' : 1000
}
cat = CatBoostClassifier(**cat_params)
cat.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 1.8568282	test: 1.8567534	best: 1.8567534 (0)	total: 32.3ms	remaining: 5m 23s
1000:	learn: 0.2797075	test: 0.2912512	best: 0.2912512 (1000)	total: 30.3s	remaining: 4m 32s
2000:	learn: 0.2003914	test: 0.2183124	best: 0.2183124 (2000)	total: 1m 1s	remaining: 4m 6s
3000:	learn: 0.1582153	test: 0.1815813	best: 0.1815813 (3000)	total: 1m 33s	remaining: 3m 38s
4000:	learn: 0.1314753	test: 0.1592465	best: 0.1592465 (4000)	total: 2m 6s	remaining: 3m 9s
5000:	learn: 0.1124700	test: 0.1441367	best: 0.1441367 (5000)	total: 2m 39s	remaining: 2m 39s
6000:	learn: 0.0982866	test: 0.1333642	best: 0.1333642 (6000)	total: 3m 12s	remaining: 2m 8s
7000:	learn: 0.0873582	test: 0.1253336	best: 0.1253336 (7000)	total: 3m 45s	remaining: 1m 36s
8000:	learn: 0.0783131	test: 0.1189993	best: 0.1189993 (8000)	total: 4m 18s	remaining: 1m 4s
9000:	learn: 0.0708305	test: 0.1139333	best: 0.1139333 (9000)	total: 4m 52s	remaining: 32.4s
9999:	learn: 0.0645231	test: 0.1097963	best: 0.1097963 (9999)	total: 5m 25

In [17]:
preds_test = np.array(cat.predict_proba(X_test))
pd.DataFrame(preds_test).to_csv('cat_boost_preds.csv')
